## Create polish dataset for revision

#### Generate folders to eth datasets

In [ ]:
eth=pl 
imp=impX_new 

# Create directories
mkdir -p $PRS_DATASETS_ELKON/bcac_onco_${eth}/${imp}
mkdir -p $PRS_DATASETS_ELKON/bcac_onco-eur-minus-${eth}/${imp}

# link ELKON to GAGA
ln -s $PRS_DATASETS_ELKON/bcac_onco_${eth} $PRS_DATASETS/bcac_onco_${eth}/${imp}
ln -s $PRS_DATASETS_ELKON/bcac_onco_eur-minus-${eth} $PRS_DATASETS/bcac_onco_eur-minus-${eth}/${imp}

#### Create pop.panel file

In [ ]:
head -n 1 $PRS_DATASETS/bcac_onco_eur/pop.panel | grep Poland > $PRS_DATASETS/bcac_onco_${eth}/pop.panel
cat $PRS_DATASETS/bcac_onco_eur/pop.panel | grep Poland >> $PRS_DATASETS/bcac_onco_${eth}/pop.panel

head -n 1 $PRS_DATASETS/bcac_onco_eur/pop.panel | grep -v Poland > $PRS_DATASETS/bcac_onco_eur-minus-${eth}/pop.panel
cat $PRS_DATASETS/bcac_onco_eur/pop.panel | grep Poland >> $PRS_DATASETS/bcac_onco_eur-minus-${eth}/pop.panel

#### Create pheno files

In [ ]:
# Create pop 


# Create pheno to eth
awk '{if(NR==FNR){c[$1]++; next;} if(c[$1]>0){print $0}}' <(cat $PRS_DATASETS/bcac_onco_pl/pop.panel) <(cat $PRS_DATASETS/bcac_onco_eur/pheno) > /specific/netapp5/gaga/gaga-pd/prs_data/datasets/dec/bcac_onc
o_pl/pheno

# Create pheno to eur-minus-eth
awk '{if(NR==FNR){c[$1]++; next;} if(c[$1]>0){print $0}}' <(cat $PRS_DATASETS/bcac_onco_eur-minus-${eth}/pop.panel) <(cat $PRS_DATASETS/bcac_onco_eur/pheno) > /specific/netapp5/gaga/gaga-pd/prs_data/datasets/dec/bcac_onco_eur-minus-${eth}/pheno

#### Generate eth dataset

In [ ]:
eth=pl 
imp=impX_new 
plink --bfile $PRS_DATASETS/${imp}/ds --keep $PRS_DATASETS/bcac_onco_${eth}/pop.panel --make-bed --out $PRS_DATASETS/bcac_onco_pl/${imp}/ds

#### QC eth set 

In [ ]:
bash qc_target_data.sh --target bcac_onco_pl --imp impX_new

#### Create reps and folds 

In [ ]:
eth=pl 
python create_cv_repetitions.py --dataset bcac_onco_${eth}

#### Create plink files for each rep and each fold (Do it for rep 1..6)

In [ ]:
base_rep=105; 
rep=1
eth=pl
imp=impX_new

for a in {1..5}; do bash prepare_cv_datasets.sh --discovery test --target bcac_onco_${eth}  --imp ${imp} --cv ${a}_5 --rep ${base_rep}_${rep}; done
bash prepare_cv_datasets.sh --discovery test --target bcac_onco_${eth}  --imp ${imp} --cv 5 --rep ${base_rep}_${rep};

#### Generate eur-minus-eth

In [ ]:
eth=pl 
imp=impX_new
plink --bfile $PRS_DATASETS_ELKON/bcac_onco_eur/${imp}/ds --remove $PRS_DATASETS_ELKON/bcac_onco_${eth}/pop.panel  --make-bed --geno 0.1 --mind 0.1 --maf 0.05 --out $PRS_DATASETS/bcac_onco_eur-minus-${eth}/${imp}/ds.QC

In [ ]:
eth=pl  
bash calc_pca2_full.sh --target bcac_onco_eur-minus-${eth} --imp impX_new --stage 2